# **MIP 연속변수사용**
- 재고증가반영 MIP 모델, 최종 프로그램

In [ ]:
# 필요한 라이브러리 설치
!pip install pulp openpyxl

import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpContinuous
import warnings
from openpyxl import Workbook

# 파일 경로
input_file = '/content/sample_data/NW_MasterData.xlsx'
output_file = '/content/sample_data/OptimizationResult수정MIP.xlsx'

# 데이터 로드
city_df = pd.read_excel(input_file, sheet_name='City')
cost_df = pd.read_excel(input_file, sheet_name='Cost')
distance_df = pd.read_excel(input_file, sheet_name='Distance')

# 상수 정의
inventory_turnover = 12
fixed_distance = 16
num_selected_wh = 3

# 도시 및 창고 목록
cities = city_df['City'].tolist()
warehouses = cost_df['WH'].tolist()

# 거리 딕셔너리 생성
distance_dict = {(row['FromCity'], row['ToCity']): row['Distance'] for _, row in distance_df.iterrows()}

# 변수 초기화
transport_cost = {}
inbound_cost = {}

for city_row in city_df.itertuples():
    city = city_row.City
    demand = city_row.Cargo
    for wh_row in cost_df.itertuples():
        wh = wh_row.WH
        dist = distance_dict.get((wh, city), fixed_distance if wh == city else None)
        if dist is None:
            warnings.warn(f"Distance missing: {wh} to {city}")
            continue
        t_cost = demand * dist * wh_row.TruckUnitRate
        i_cost = demand * wh_row.InboundRate
        transport_cost[(city, wh)] = t_cost
        inbound_cost[(city, wh)] = i_cost

# 모델 정의
model = LpProblem("Warehouse_Optimization", LpMinimize)

# 변수 정의
x = LpVariable.dicts("assign", (cities, warehouses), cat=LpBinary)
y = LpVariable.dicts("select", warehouses, cat=LpBinary)
z = LpVariable.dicts("total_demand", warehouses, lowBound=0, cat=LpContinuous)

# 목적함수 정의 (연속변수 z[w] 기반 창고비 반영)
model += (
    lpSum([
        (transport_cost[(c, w)] + inbound_cost[(c, w)]) * x[c][w]
        for c in cities for w in warehouses if (c, w) in transport_cost
    ]) +
    lpSum([
        z[w] * cost_df[cost_df['WH'] == w]['WHUnitRate'].values[0] * 12 * np.sqrt(num_selected_wh) / inventory_turnover
        for w in warehouses
    ])
)

# 제약조건
# (1) 각 도시는 하나의 창고에만 할당
for c in cities:
    model += lpSum([x[c][w] for w in warehouses if (c, w) in transport_cost]) == 1

# (2) 창고는 정확히 3개 선택
model += lpSum([y[w] for w in warehouses]) == num_selected_wh

# (3) 도시는 선택된 창고에만 할당 가능
for c in cities:
    for w in warehouses:
        if (c, w) in transport_cost:
            model += x[c][w] <= y[w]

# (4) 연속 변수 z[w] 정의: 창고별 총 수요량
for w in warehouses:
    model += z[w] == lpSum([
        city_df[city_df['City'] == c]['Cargo'].values[0] * x[c][w]
        for c in cities if (c, w) in transport_cost
    ])

# 최적화 수행
model.solve()

# 결과 저장용 리스트 초기화
city_results = []
wh_summary = {w: {'Cargo': 0, 'Transport': 0, 'Inbound': 0, 'Storage': 0} for w in warehouses}

# 도시별 결과 정리
for c in cities:
    for w in warehouses:
        if (c, w) in transport_cost and x[c][w].varValue == 1:
            demand = city_df[city_df['City'] == c]['Cargo'].values[0]
            t = transport_cost[(c, w)]
            i = inbound_cost[(c, w)]
            wh_summary[w]['Cargo'] += demand
            wh_summary[w]['Transport'] += t
            wh_summary[w]['Inbound'] += i
            city_results.append([c, demand, t + i, w, t, i, 0])  # 임시 Storage = 0
            break

# 창고 단위 창고비 계산
for w in warehouses:
    if y[w].varValue == 1:
        cargo = wh_summary[w]['Cargo']
        unit_rate = cost_df[cost_df['WH'] == w]['WHUnitRate'].values[0]
        wh_summary[w]['Storage'] = (cargo / inventory_turnover) * unit_rate * 12 * np.sqrt(num_selected_wh)

# city_results의 StorageCost와 TotalCost 재계산
for row in city_results:
    wh = row[3]
    cargo_share = row[1] / wh_summary[wh]['Cargo'] if wh_summary[wh]['Cargo'] else 0
    storage_contrib = wh_summary[wh]['Storage'] * cargo_share
    row[6] = storage_contrib
    row[2] += storage_contrib

# 시트별 데이터프레임 생성
city_df_out = pd.DataFrame(city_results, columns=['City', 'Cargo', 'TotalCost', 'Warehouse', 'TransportCost', 'InboundCost', 'StorageCost'])
wh_df_out = pd.DataFrame([
    [w, v['Cargo'], v['Transport'], v['Inbound'], v['Storage'], v['Transport'] + v['Inbound'] + v['Storage']]
    for w, v in wh_summary.items() if y[w].varValue == 1
], columns=['Warehouse', 'TotalCargo', 'TransportCost', 'InboundCost', 'StorageCost', 'TotalCost'])

summary_df = pd.DataFrame({
    '항목': ['운송비', '수입물류비', '창고비', '총 물류비'],
    '합계': [
        city_df_out['TransportCost'].sum(),
        city_df_out['InboundCost'].sum(),
        city_df_out['StorageCost'].sum(),
        city_df_out['TotalCost'].sum()
    ]
})

# 결과 출력
from IPython.display import display
print("\n[Summary Sheet]")
display(summary_df)

print("\n[WH Sheet]")
display(wh_df_out)

print("\n[City Sheet - New YorkNY]")
display(city_df_out[city_df_out['City'] == 'New YorkNY'][['TransportCost', 'InboundCost', 'StorageCost', 'TotalCost']])

# 결과 엑셀 저장
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    city_df_out.to_excel(writer, sheet_name='City', index=False)
    wh_df_out.to_excel(writer, sheet_name='WH', index=False)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.2 MB/s eta 0:00:00

[Summary Sheet]


,항목,합계
0,운송비,9.627102e+06
1,수입물류비,3.465766e+07
2,창고비,5.862423e+06
3,총 물류비,5.014719e+07



[WH Sheet]


,Warehouse,TotalCargo,TransportCost,InboundCost,StorageCost,TotalCost
0,Los AngelesCA,3100623,3.830358e+06,7.751558e+06,2.631890e+06,1.421381e+07
1,HoustonTX,2407766,2.536167e+06,1.039717e+07,1.381665e+06,1.431500e+07
2,ColumbusOH,3228414,3.260576e+06,1.650894e+07,1.848867e+06,2.161838e+07



[City Sheet - New YorkNY]


,TransportCost,InboundCost,StorageCost,TotalCost
0,1.083639e+06,4.222447e+06,472879.94053,5.778966e+06
